In [1]:
%matplotlib inline
import matplotlib
from bqplot import pyplot as plt
import ipywidgets
import bqplot
import numpy as np
import pandas as pd
import datetime as dt
building_dataset = pd.read_csv(r"C:\Users\23137\Desktop\445\DataViz\building_inventory.csv", 
                               na_values = {'Year Acquired': 0,  'Year Constructed': 0,  'Square Footage': 0})
building_dataset


,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975.0,1975.0,144.0,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8857,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte. 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,NaN,NaN,432.0,1,0,0,Storage,NaN,NaN
8858,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,NaN,NaN,330.0,1,0,0,Storage,NaN,NaN
8859,Department of Transportation,Quincy Maintenance Storage Facility,800 Koch's Lane,Quincy,62305,Adams,18,Darin M. LaHood,94,Frese Randy E.,...,In Use,NaN,1987.0,130.0,1,0,0,Storage,High Hazard,NaN
8860,Illinois Community College Board,Illinois Valley Community College - Oglesby,815 North Orlando Smith Avenue,Oglesby,61348,LaSalle,16,Adam Kinzinger,76,Long Jerry Lee,...,In Use,1971.0,1971.0,49552.0,1,1,0,Education,Education,Not provided


In [2]:
chart = building_dataset.groupby(['Agency Name', 'Congress Dist', 'Year Acquired'], as_index=False)['Square Footage'].sum()
chart
table = pd.pivot_table(chart, values=['Square Footage'], index=['Congress Dist'],
                    columns=['Agency Name' ], aggfunc=np.sum)


In [3]:
mySelectedLabel = ipywidgets.Label()

In [4]:
col_sc = bqplot.ColorScale(scheme="Reds") 
# this is because the "bins" are just bins -- their order is NOT numerically important
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# 3. Axis -- for colors, the axis is a colorbar!
ax_col = bqplot.ColorAxis(scale = col_sc, orientation='vertical', 
                          side='right') 
ax_x = bqplot.Axis(scale = x_sc, orientation='horizontal',tick_rotate = 270,
                   tick_style ={'font-size': 6}, label = 'Agency Names') # same x/y ax we had before
ax_y = bqplot.Axis(scale = y_sc, orientation='vertical', label='Congressional District')

anames = table.columns.levels[1].to_list()
for i in range(len(anames)):
    anames[i] = '      ' + anames[i]


# 4. Mark -- heatmap -- CHANGE HERE
heat_map = bqplot.GridHeatMap(color = np.log10(table.values),
                              row = table.index,
                              column = anames,
                              scales = {'color':col_sc, 
                                        'row':y_sc, 'column':x_sc},
                             interactions={'click':'select'},
                             anchor_style={'fill':'blue'},
                             selected_style={'opacity':1.0},
                             unselected_style={'opacity':0.9})

In [5]:
i,j = 2,16

mask = (chart['Congress Dist']== table.index[i]) & (chart['Agency Name']== table.columns.levels[1][j])
subset= chart[mask]
grouped = subset.groupby("Year Acquired")["Square Footage"].sum()

x_sch = bqplot.LinearScale() # range of z-axis data
y_sch = bqplot.LinearScale() # frequency of z-axis data in bins

# (2) axis: x/y
x_axh = bqplot.Axis(scale = x_sch, label = 'Year Acquired')
y_axh = bqplot.Axis(scale = y_sch,  
                    orientation = 'vertical', 
                    label='Square Footage')

line_plot = bqplot.pyplot.plot(x = grouped.index,
                               y = grouped.values, 
                              scales={'x':x_sch, 'y':y_sch})

In [6]:
grouped

Year Acquired
1930.0     69203.0
1934.0     33605.0
1935.0     85292.0
1936.0    184416.0
1945.0      1624.0
1947.0      7831.0
1955.0       140.0
1957.0       336.0
1958.0       864.0
1959.0      1266.0
1965.0      2370.0
1967.0      6046.0
1985.0     16976.0
1986.0      3000.0
1987.0      1128.0
2001.0     36414.0
2003.0     27500.0
2008.0       150.0
2009.0       900.0
Name: Square Footage, dtype: float64

In [7]:
def get_data_value(change):
    if len(change['owner'].selected) == 1: #only 1 selected
        i,j = change['owner'].selected[0]
        v = table.iloc[i,j]
        v2 = i,j
        mySelectedLabel.value = 'Sum of total square footage = ' + str(v) + f" {str(v2)}"# set our label
        mask = ((chart['Congress Dist']== table.index[i]) & (chart['Agency Name']== table.columns.levels[1][j]))
        subset= chart[mask]
        grouped = subset.groupby("Year Acquired")["Square Footage"].sum()

        line_plot.x = grouped.index
        line_plot.y = grouped.values
heat_map.observe(get_data_value, 'selected')


In [8]:
fig_heatmap = bqplot.Figure(marks = [heat_map], axes=[ax_col, ax_x, ax_y])
fig_line = bqplot.Figure(marks = [line_plot], axes = [x_axh, y_axh])

In [9]:
fig_heatmap.layout.min_width='500px'
fig_line.layout.min_width='500px'

myDashboard = ipywidgets.VBox([mySelectedLabel, ipywidgets.HBox([fig_heatmap,fig_line])])
myDashboard

In [10]:
v = table.iloc[2,16]

Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(scheme='Reds'), side='right'), Axis(label='Age…

In [12]:

type(str(v))

str